Configurar Google Translate API e baixar o json
https://cloud.google.com/translate/docs/basic/setup-basic

In [ ]:
%mv ./Translation*.json ./key.json

Baixa MilkQA para tradução

In [0]:
!wget https://www.dropbox.com/s/1ozzofw3f7thq3g/MilkQA-v2.tar.gz
!tar -xzvf MilkQA-v2.tar.gz
%cd MilkQA-v2/data

Pegar as questões, assuntos e respostas e passar para lista de palavras a serem traduzidas

In [0]:
import os
import re
from pathlib import Path

files = os.listdir(os.curdir)
files = filter(lambda x: "question" in x, files)
numbers = [re.search("(.*)-", x).group(1) for x in files]

words = []
unsolved = []
for number in numbers:
        print(number)
        question = Path(number + "-question.txt").read_text()
        question = re.split("<.*>", question)
        question = list(filter(None, question))
        if len(question) != 2:
            unsolved.append(number)
            continue

        words.append(question[0])
        words.append(question[1])

        answer = Path(number + "-answer.txt").read_text()
        words.append(answer)

6800
7886
6409
7377
7773
6979
7861
7031
7627
7177
6420
7254
7408
7823
6761
6347
6391
7079
667
6521
6759
6382
7533
7629
6428
7574
7062
6487
6953
7561
6602
6766
7655
7024
7902
765
7308
6432
6466
6527
6828
735
7862
6793
7884
6575
6799
7376
6639
7499
6410
7375
7321
7721
7554
6805
708
6352
6660
8171
9165
8329
8818
804
8879
8940
8384
8842
7985
9119
9211
8552
8985
8785
9352
9082
8726
925
9400
8441
9074
9056
8763
8577
8128
9180
8992
8721
8447
9037
8145
8512
8521
8452
8607
9051
8723
8783
7903
9134
8906
9303
8273
8897
9196
8761
851
8482
8588
8971
8681
8932
9215
8999
8107
9106
9320
8549
8613
8982
8722
8076
9284
8697
8270
877
8414
8018
8698
8584
8769
8456
9168
8213
8443
862
9203
7939
9096
8378
8845
9163
905
8335
8596
8935
9289
8914
8444
8679
9021
8917
9032
9019
8621
8827
8244
8907
8040
80
9254
9020
8216
9107
9368
9147
9071
8828
8039
9406
8873
8524
8403
8108
8239
8587
8710
8396
8214
8299
8814
8195
8729
8506
9265
9380
9072
9220
9148
7970
7930
9075
8383
8912
8565
9285
9280
9184
9007
8072
9373
8002
82

Imprimir os exemplos mal-formatados não resolvidos

In [0]:
print(unsolved)

['25414', '13050', '13546', '1487', '16762', '10551']


Resolver a questão 13546

In [0]:
words.append("cana de açucar")
words.append("""sou de dois vizinhos , e dou assitencia técnica para 
 produtores de leite da agricultura familiar desta cidade . E gostria
    de ter algumas informações sobre cana de açucar cal pecuario na
    suplementação de vacas de leite.""")
answer = Path("13546-answer.txt").read_text()
words.append(answer)

Resolver questões 13050, 10551 e 1487: sem assunto

In [0]:
for number in [13050,10551,1487]:
        print(number)
        question = Path(str(number) + "-question.txt").read_text()
        question = re.sub("<.*>", "", question)
        print(question)
        words.append("")
        words.append(question)

        answer = Path(str(number) + "-answer.txt").read_text()
        words.append(answer)

13050


QUAL É O PRIMEIRO PASSO PARA COMEÇAR UMA CRIAÇÃO DE VACA LEITERAS NO 
NORDESTE DA BAHIA?

10551


Gostaria de informações sobre o trato de vacas em lactação, com MELAÇO
de cana.A eficiencia do produto e se poderia utilizar com uréia+sulfato
de amônia.Qual a quantidade diária por animal?Devo utilizar o em pó
diluído em que proporção?

1487

Como combater a figueira nas vacas leiterias?



Resolver questões 25414, 16762: sem delimitador \<corpo\>

In [0]:
for number in [25414,16762]:
        print(number)
        question = Path(str(number) + "-question.txt").read_text()
        question = re.sub("<.*>", "", question)
        question = re.split("\n\n", question)
        question = list(filter(None, question))

        words.append("")
        words.append(question)
        print(question)

        answer = Path(str(number) + "-answer.txt").read_text()
        words.append(answer)

25414
['\ncruzamento', 'tenho algumas vacas puras holandesas; tenho um poblema como o clima muito quente da regiao. se eu cruzar com jersey posso amenizar este probblema.abrigado.\n']
16762
['\nmoscas no gado', 'Como posso eliminar moscas que estão infestando o meu curral e estressando as vacas leiteiras. Não são moscas do chifres, mas em função do período chuvoso elas aparecem aos montes. Existe algum produto que se aplique e não prejudique o leite e os bezerros?\n']


Importar biblioteca de tradução de lista de palavras

In [0]:
!jupyter nbconvert --to python Auxiliar.ipynb

In [0]:
import Auxiliar
Auxiliar.translate_lst2dct(words, 'en', 'milkqa-dic.json')